### Импорт библиотек

In [81]:
import spacy
import pandas as pd
import nltk
import numpy as np
import re
from nltk.corpus import stopwords
from tqdm import tqdm
from collections import Counter, defaultdict
import itertools
from spacy.symbols import NOUN, PROPN, PUNCT, SYM, ADP, DET, ADJ

nlp = spacy.load("ru_core_news_lg")
stopwords_ru = stopwords.words("russian")
tqdm.pandas(desc="progress-bar")

### Предобработка данных

In [94]:
df = pd.read_excel('review.xlsx')
df = df.drop('Unnamed: 0', axis=1)

In [95]:
df.head(5)

,review
0,"День 8-го марта прошёл, можно и итоги подвести..."
1,Отмечали в этом ресторане день рождение на пер...
2,Хочу поделиться своим впечатлением от посещени...
3,Добрый день! Были вчера с друзьями в этом кафе...
4,Отметили с мужем годовщину свадьбы 6 ноября в ...


In [96]:
df['review'][1]

'Отмечали в этом ресторане день рождение на первом этаже в субботу вечером. Хочу выразить большую благодарность прежде всего руководству ресторана, обслуживающему персоналу и конечно же тем сотрудникам, которые готовят посетителям заведения столь вкусные блюда. Понравилось абсолютно все. Конечно же впечатляет интерьер: уютно и красиво. Стол был шикарный. Нас обслуживал официант Бахрам, спасибо ему. Хочу отметить, что для ресторана такого уровня цены не высокие.'

In [97]:
def sent_token(text):
    """
    Токенизация по предложениям
    """
    text = str(text).lower()
    text = text.replace('...', '.').replace('.', '. ')
    return [nlp(x, disable=["ner"]) for x in nltk.sent_tokenize(text)]

In [98]:
df["sentences"] = df["review"].progress_map(lambda x: sent_token(x))

progress-bar: 100%|██████████| 284/284 [00:31<00:00,  9.00it/s]


In [99]:
df = df.explode('sentences')
df.index = np.arange(0, len(df))

In [103]:
def process_sent(text, remove_stopwords=False, min_token_length=3):
    """ 
    Apply text preprocessing steps 
    """
    if remove_stopwords:
        return " ".join([token.lemma_.lower() for token in text if not token.is_stop 
                         and len(token.text) >= min_token_length])
    else:
        return " ".join([token.lemma_.lower() for token in text if len(token.text) >= min_token_length])

In [104]:
df['clean_sent'] = df['sentences'].progress_map(lambda x: process_sent(x))

progress-bar: 100%|██████████| 3466/3466 [00:00<00:00, 43614.77it/s]


### Извлечение аспектных терминов

In [134]:
def get_noun_phrases(texts):
    
    features_dict = Counter()
    
    for item in tqdm(texts):
        text_nlp = nlp(item)
        count = 0
        half_chunk = ""
        for word in text_nlp:
            if word.tag_ in('NOUN', 'PROPN'):
                count+=1
                if count>=1:
                    half_chunk = half_chunk + str(word) + " "
            else:
                half_chunk = half_chunk + "---"
                count = 0
        half_chunk = re.sub(r"-+","?",half_chunk).split("?")
        half_chunk = [x.strip() for x in half_chunk if x!=""]
        
        half_chunk = [x for x in half_chunk if x != '']
    
        features_dict.update(half_chunk)
    return features_dict

In [135]:
features_dict = get_noun_phrases(list(df['clean_sent']))

100%|██████████| 3466/3466 [00:42<00:00, 81.66it/s] 


In [136]:
features_dict.most_common()

[('ресторан', 224),
 ('место', 153),
 ('кухня', 131),
 ('раз', 131),
 ('интерьер', 124),
 ('заведение', 117),
 ('обслуживание', 112),
 ('меню', 103),
 ('блюдо', 99),
 ('столик', 92),
 ('официант', 92),
 ('еда', 75),
 ('заказ', 74),
 ('стол', 74),
 ('вечер', 73),
 ('минута', 71),
 ('зал', 71),
 ('время', 68),
 ('человек', 63),
 ('спасибо', 61),
 ('порция', 58),
 ('впечатление', 56),
 ('гость', 55),
 ('цена', 55),
 ('персонал', 54),
 ('музыка', 54),
 ('пиво', 47),
 ('день', 45),
 ('официантка', 45),
 ('салат', 44),
 ('отзыв', 41),
 ('атмосфера', 41),
 ('девушка', 37),
 ('десерт', 35),
 ('компания', 33),
 ('уровень', 32),
 ('праздник', 27),
 ('выбор', 27),
 ('напиток', 27),
 ('год', 27),
 ('друг', 25),
 ('минус', 25),
 ('банкет', 25),
 ('сервис', 24),
 ('вкус', 24),
 ('обстановка', 24),
 ('дело', 23),
 ('день рождение', 23),
 ('народ', 23),
 ('подруга', 23),
 ('мясо', 23),
 ('тарелка', 22),
 ('вход', 22),
 ('час', 22),
 ('восторг', 21),
 ('настроение', 21),
 ('ребёнок', 21),
 ('этаж', 20)

In [179]:
def feature_pruning(features_dict, features_list=[], min_support=3, min_percent=0.01, verbose=False):
    """
    
    """
    # Apply p-support pruning
    result = []
    pruned_features_dict = features_dict.copy()
    for k1, v1 in features_dict.items():
        # Find single word features and the support
        if len(k1.split())==1:
            p_support = v1
            if verbose:
                print(f'feature {k1} support={p_support}')
            # Calculate p_support by subtracting the number of times the single noun appears in other
            #   noun_phrases
            for k2, v2 in features_dict.items():
                if len(k2.split())!=1:
                    if k1 in k2:
                        p_support -= v2
                        if verbose:
                            print(f'feature {k1} found in {k2}, updated support={p_support}') 
            # If the final p_support value < min_support, remove this feacture
            if p_support < min_support:
                pruned_features_dict.pop(k1, None)
                if verbose:
                    print(f'feature {k1} removed, support={p_support}')
              
    # Apply frequent feature pruning
    features = []
    for k, v in pruned_features_dict.items():
        if v > int(len(df['clean_sent']) * min_percent):
            result.append(k)
            
    return result, pruned_features_dict

In [180]:
result, pruned_features_dict = feature_pruning(features_dict)

In [181]:
result

['отзыв',
 'ресторан',
 'столик',
 'заказ',
 'человек',
 'блюдо',
 'меню',
 'обслуживание',
 'интерьер',
 'вечер',
 'кухня',
 'персонал',
 'стол',
 'гость',
 'музыка',
 'спасибо',
 'раз',
 'заведение',
 'атмосфера',
 'место',
 'десерт',
 'цена',
 'впечатление',
 'еда',
 'пиво',
 'минута',
 'порция',
 'время',
 'официантка',
 'девушка',
 'зал']